In [19]:
! pip install transformers
! pip install datasets
from datasets import load_dataset
import random
random.seed(42)

In [20]:
train_data = load_dataset("yangwang825/rte", split="train")
dev_data = load_dataset("yangwang825/rte", split="validation")
test_data = load_dataset("yangwang825/rte", split="test")

In [21]:
print(train_data)

Dataset({
    features: ['text1', 'text2', 'label', 'idx', 'label_text'],
    num_rows: 2490
})


In [22]:
from transformers import BertTokenizer,BertModel

In [23]:
tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-tiny")
model = BertModel.from_pretrained("prajjwal1/bert-tiny")

In [24]:
train_tokens = tokenizer(train_data["text1"], train_data["text2"], truncation=True, padding=True,max_length=512, return_tensors="pt")
dev_tokens = tokenizer(dev_data["text1"], dev_data["text2"], truncation=True, padding=True,max_length=512, return_tensors="pt")
test_tokens = tokenizer(test_data["text1"], test_data["text2"], truncation=True, padding=True,max_length=512, return_tensors="pt")

In [25]:
import torch

In [26]:
from torch.utils.data import DataLoader, TensorDataset
train_dataset = TensorDataset(train_tokens["input_ids"], train_tokens["token_type_ids"], train_tokens["attention_mask"], torch.tensor(train_data["label"]))
dev_dataset = TensorDataset(dev_tokens["input_ids"], dev_tokens["token_type_ids"], dev_tokens["attention_mask"], torch.tensor(dev_data["label"]))
test_dataset = TensorDataset(test_tokens["input_ids"], test_tokens["token_type_ids"], test_tokens["attention_mask"], torch.tensor(test_data["label"]))
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [27]:
from transformers import BertModel
import numpy as np

In [28]:
import torch
class CustomBert(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.bert = BertModel.from_pretrained("prajjwal1/bert-tiny")        # Invoke the constructor of the parent class
    self.l1 = torch.nn.Linear(128, 2,bias = True)

  def forward(self, inp_ids,tok_typs,att_masks):
    a = self.bert(input_ids=inp_ids, token_type_ids=tok_typs,attention_mask=att_masks)
    z = self.l1(a.pooler_output)
    return z

device = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")
model = CustomBert().to(device)
# print(model)
for param in model.bert.parameters():
            param.requires_grad = False
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [29]:
num_epochs = 10
best_perf_dict = {"metric": 0, "epoch": 0}
for epoch in range(1,num_epochs+1):
  print(f"\nEpoch {epoch}")
  accuracies = []
  loss_mat =[]
  for batch in train_dataloader:
    model.train()
    optimizer.zero_grad()
    input_ids,token_type_ids,attention_mask,labels = batch
    outputs = model(input_ids,token_type_ids,attention_mask)
    pred_lab = torch.argmax(outputs,dim=1)
    loss = loss_fn(outputs,labels)
    loss.backward() # computing the gradients
    optimizer.step()  # Performs the optimization
    temp_accuracy = 0
    for k in range(len(pred_lab)):
      if pred_lab[k] == labels[k]:
        temp_accuracy = temp_accuracy + 1
    temp_accuracy = (temp_accuracy)/(len(pred_lab))
    accuracies.append(temp_accuracy)
    loss_mat.append(loss.detach().numpy())
  print(f"Average Accuracy: {np.mean(accuracies)}")
  print(f"Average Loss: {np.mean(loss_mat)}")
    # Evaluation Loop
  dev_accuracies = []
  for batch in dev_dataloader:
      model.eval()
      input_ids,token_type_ids,attention_mask,labels = batch
      with torch.no_grad():
          # out = model(inp.to(device))
          outputs = model(input_ids,token_type_ids,attention_mask)
          pred_lab = torch.argmax(outputs,dim=1)
          temp_accuracy = 0
          for k in range(len(pred_lab)):
            if pred_lab[k] == labels[k]:
              temp_accuracy = temp_accuracy + 1
          temp_accuracy = (temp_accuracy)/(len(pred_lab))
          dev_accuracies.append(temp_accuracy)
  print(f"Dev Accuracy: {np.mean(dev_accuracies)}")
  dev_f1 = np.mean(dev_accuracies)
  # Update the `best_perf_dict` if the best dev performance seen
  # so far is beaten
  if dev_f1 > best_perf_dict["metric"]:
      best_perf_dict["metric"] = dev_f1
      best_perf_dict["epoch"]  = epoch
      torch.save({
          "model_param": model.state_dict(),
          "optim_param": optimizer.state_dict(),
          "dev_metric": dev_f1,
          "epoch": epoch
      }, f"./models/{epoch}")



Epoch 1
Average Accuracy: 0.5260416666666666
Average Loss: 0.6920291781425476
Dev Accuracy: 0.5583333333333333

Epoch 2
Average Accuracy: 0.5336538461538461
Average Loss: 0.688162088394165
Dev Accuracy: 0.5402777777777777

Epoch 3
Average Accuracy: 0.5435096153846154
Average Loss: 0.6852706670761108
Dev Accuracy: 0.5402777777777777

Epoch 4
Average Accuracy: 0.5354166666666667
Average Loss: 0.6885973215103149
Dev Accuracy: 0.5402777777777777

Epoch 5
Average Accuracy: 0.5509615384615385
Average Loss: 0.6835043430328369
Dev Accuracy: 0.5395833333333333

Epoch 6
Average Accuracy: 0.5568910256410257
Average Loss: 0.6843078136444092
Dev Accuracy: 0.5541666666666667

Epoch 7
Average Accuracy: 0.554326923076923
Average Loss: 0.6839386224746704
Dev Accuracy: 0.5472222222222222

Epoch 8
Average Accuracy: 0.563301282051282
Average Loss: 0.6830799579620361
Dev Accuracy: 0.5756944444444445

Epoch 9
Average Accuracy: 0.5711538461538461
Average Loss: 0.6797413229942322
Dev Accuracy: 0.561111111111

In [30]:
best_epoch = best_perf_dict["epoch"]
model_path = f"./models/{best_epoch}"
checkpoint = torch.load(model_path)
test_accuracies = []
for batch in test_dataloader:
    model.eval()
    input_ids,token_type_ids,attention_mask,labels = batch
    with torch.no_grad():
        # out = model(inp.to(device))
        outputs = model(input_ids,token_type_ids,attention_mask)
        pred_lab = torch.argmax(outputs,dim=1)
        temp_accuracy = 0
        for k in range(len(pred_lab)):
          if pred_lab[k] == labels[k]:
            temp_accuracy = temp_accuracy + 1
        temp_accuracy = (temp_accuracy)/(len(pred_lab))
        test_accuracies.append(temp_accuracy)
print(f"Test Accuracy: {np.mean(test_accuracies)}")

Test Accuracy: 0.5694444444444444
